In [ ]:
# import package
import os
import pandas as pd
import ydata_profiling  # ydata_profiling 패키지를 활용함

# age_gorup function (연령 그룹을 분류하는 함수)
def chk_age(x):
    x = str(x)
    x = x.replace('대','')
    x = x.replace('1020','10/20')
    x = x.replace('2030','20/30')
    x = x.replace('3040','30/40')
    x = x.replace('4050','40/50')
    x = '/'.join(set(x.split('/')))  # 중복제거 (dedupication)
    
    if '대학생' in x or '신입생' in x:
        return '20'
    elif '학생' in x:
        return '10'
    elif '직장인' in x:
        return '20/30/40/50'
    elif x == 'nan':
        return float('nan')
    else:
        return x
    
# weight_category function (무게 그룹을 분류하는 함수)
def chk_weight(x):
    x = str(x)
    x = x.replace('경량','가벼운')
    x = x.replace('묵직한','무거운')
    x = '/'.join(set(x.split('/')))  # 중복제거 (deduplication)
    
    if x == 'nan':
        return float('nan')
    else:
        return x
    
# season_category function (계절 그룹을 분류하는 함수)
def chk_season(x):
    x = str(x)
    x = '/'.join(set(x.split('/')))  # deduplication
    
    if x == 'nan':
        return float('nan')
    elif '봄' in x:
        return '봄'
    elif '여름' in x or '휴가' in x:
        return '여름'
    elif '겨울' in x:
        return '겨울'
    elif '가을' in x:
        return '가을'
    else:
        return x

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# CSV파일 전체 가져오기
filelist = []

for (path, dirs, files) in os.walk('./'):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.csv':  # csv file listup
            filelist.append("%s/%s" % (path, filename))


In [8]:
# CSV 각 파일 읽은 뒤 병합하기
df_merge = pd.read_csv(filelist[0])

for i in filelist[1:]:
    try:
        df2 = pd.read_csv(filelist[1])  # encoding check
    except UnicodeDecodeError:
        df2 = pd.read_csv(filelist[1], encoding='cp949')  # encoding check

    df_merge = pd.concat([df_merge, df2])

In [9]:
# data grouping
# 'category_middle_desc' 열의 값을 기반으로 'top_and_bottom'이라는 새로운 열을 만듦. 
# 'category_middle_desc'의 값을 {'원피스':'한벌옷','팬츠':'하의','아우터':'상의','상의':'상의','스커트':'하의'}와 매핑함.
df_merge['top_and_bottom'] = df_merge['category_middle_desc'].map({'원피스':'한벌옷','팬츠':'하의','아우터':'상의','상의':'상의','스커트':'하의'})
df_merge['age_group'] = df_merge['age'].apply(lambda x: chk_age(x))
df_merge['weight_category'] = df_merge['weight'].apply(lambda x: chk_weight(x))
df_merge['season_category'] = df_merge['season'].apply(lambda x: chk_season(x))

df_target = df_merge[['top_and_bottom','category_small_desc','color','material','print_pattern','characteristic','detail','age_group','weight_category','season_category','sensibility','sale_price','recent_sale_count']]

# 전처리 파일 추출하기
df_merge.to_csv('./preprocessing.csv', index=False)

In [ ]:
# EDA
# 데이터프레임의 프로파일 보고서 생성
pf = df_target.profile_report(
    correlations=None,  # disable correlations for large datasets
    missing_diagrams={
      "heatmap": False,  # disable heatmap for large datasets
    }
)
# HTML 파일로 저장
pf.to_file('./report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]